In [1]:
import pandas as pd
import zipfile
import os
import time
from google.colab import drive
from PIL import Image

In [2]:
!pip install pysrt
import pysrt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pysrt: filename=pysrt-1.1.2-py3-none-any.whl size=13443 sha256=13d53554760b746a85c3bd905128f2e16ac81b41f6292355c763171085b29e3d
  Stored in directory: /root/.cache/pip/wheels/30/7f/e8/55de9a9b07302d9e7fe47c27910e3bea0c48536153e74bd7e6
Successfully built pysrt


In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# чтобы разгрузить код я буду путь к файлам сохранять в эту переменную
PATH = None

In [5]:
# создаем датафрейм
PATH = '/content/drive/MyDrive/Masterskaya_2/English_scores.zip (Unzipped Files)/movies_labels.xlsx'
data = pd.read_excel(PATH)
data.head()

FileNotFoundError: ignored

In [ ]:
# читаем файл с субтитрами
PATH = '/content/drive/MyDrive/Masterskaya_2/English_scores.zip (Unzipped Files)/Subtitles_all/Subtitles/10_Cloverfield_lane(2016).srt'
subs = pysrt.open(PATH)

In [ ]:
#help (pysrt)

In [ ]:
# я посмотрела как будут выглядеть субтитры
subtitle_texts = [subtitle.text for subtitle in subs]
#subtitle_texts

Для того, чтобы получить субтитры в удобноваримом формате, мы пройдемся циклом по всему содержимому папки Subtitles, которая содержит папки с названиями фильмов. Для этого используем модуль os, который позволяет работать с файлами и директориями.

In [ ]:
# создадим пустые списки в которые соберем название и субтитры каждого фильма
subtitles = []
movie = []

Я выбирала между сохранить в словарь название:субтитры или два списка.
Решила сделать два списка, потому что потом мне нужно будет сравнивать фильмы в дф и в списке фильмов с субтитрами, а это легче сделать из списка, а потом добавить в дф.

In [ ]:
PATH = '/content/drive/MyDrive/Masterskaya_2/English_scores.zip (Unzipped Files)/Subtitles_all/Subtitles'

In [ ]:
# сохраним названия файлов в папке Subtitles в список
film_list = os.listdir(PATH)
film_list[:5]

Обращаем внимание, что имеются строки в которых точка используется как разделитель, это важно, так как нам нужно сохранить названия фильмов без расширения .srt

---

Делить будем так:
- находим индекс где стоит **последняя** точка
- добавляем строку до этого индекса

In [ ]:
# проверяем
last_dot = film_list[2].rfind('.')
film_list[2][:last_dot]

In [ ]:
# создаем цикл, который будет проходить по элементам списка и сохранять название для кажлого фильма
for film in film_list:
    last_dot = film.rfind('.')
    movie.append(film[:last_dot])
movie[:10]

В работе цикла возникла ошибка что файл не найден в указанной директории.
Я попробовала открыть отдельно этот файл(вручную прописав путь к нему) и возникла ошибка кодировки. Чтобы решить эти вопросы:
- Поэтому добавила в цикл try-exept, чтобы обработать все возможные кодировки
- Создала переменную путь и название, чтобы избежать того, что возникает ошибка из-за некорректной обработки названий.


In [ ]:
# создаем цикл, который будет проходить по элементам списка и сохранять субтитры
for film in film_list:
  file_to_read = os.path.join(PATH, film)
  subtitles.append(pysrt.open(file_to_read, encoding='iso-8859-1'))

In [ ]:
len(subtitles)

In [ ]:
len(movie)

In [ ]:
movie.count('None')

Теперь можно обработать названия фильмов, чтобы добавить субтитры к фильмам.
Обрабатывать буду следующими шагами:
- все слова к строчному регистру
- в качестве разделителя только знак нижнего подчеркивания "_"
- удалю все пробелы, знаки между словами, оставлю только цифры.

In [ ]:
!pwd


In [ ]:
!git init

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/drive/MyDrive/Masterskaya_2/.git/


In [ ]:
!git clone https://github.com/AV-BOLT/Masterskaya_2.git

fatal: destination path 'Masterskaya_2' already exists and is not an empty directory.


In [ ]:
!git remote add origin https://github.com/AV-BOLT/Masterskaya_2.git

In [ ]:
!git status

On branch master

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	English_scores.zip
	English_scores.zip (Unzipped Files)/
	Masterskaya_2/
	Notebooks/

nothing added to commit but untracked files present (use "git add" to track)


In [ ]:
ls


 English_scores.zip                      Masterskaya_2/
'English_scores.zip (Unzipped Files)'/   Notebooks/


In [ ]:
!git commit -m 'Создала файл ноутбука'

[master 8fada91] Создала файл ноутбука
 1 file changed, 1 insertion(+), 1 deletion(-)


In [ ]:
!git config --global user.email "gelika176@.com"
!git config --global user.name "Anzhelika Boltneva"

In [ ]:
!git add .

In [ ]:
!git commit -m 'Создала новый репозиторий и перезалила ноутбук'

[master (root-commit) 3c7ba11] Создала новый репозиторий и перезалила ноутбук
 1 file changed, 1 insertion(+)
 create mode 100644 determine_langlevel_by_subs.ipynb


In [ ]:
!git push origin master

fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
!git status

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   determine_langlevel_by_subs.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git add determine_langlevel_by_subs.ipynb

In [ ]:
!git status

:wq
On branch master
Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   determine_langlevel_by_subs.ipynb

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   determine_langlevel_by_subs.ipynb



In [ ]:
!pwd

'/content/drive/MyDrive/Masterskaya_2/Notebooks'

# Новый раздел